In [1]:
import sys, os
import requests
import gradio as gr
from PIL import Image, ImageDraw

In [2]:
current_dir = os.getcwd()
parent_dir = os.path.dirname(current_dir)
sys.path.append(parent_dir)

In [3]:
from common.config import get_config_info
from common.image import random_color, get_font

In [4]:
config = get_config_info('./config/settings1.json')

FEATURES = ['read', 'smartCrops', 'tags', 'people', 'caption', 'denseCaptions', 'objects']
ENDPOINT = config['cv_endpoint']
MODEL = config['cv_api_model']
API_KEY = config['cv_api_key']
API_VERSION = config['cv_api_version']

In [5]:
# 테스트 코드
KO_FEATURES = ['read', 'smartCrops', 'tags', 'people']
selected_feature_list = ['tags', 'people', 'caption', 'denseCaptions']
list(set(KO_FEATURES) & set(selected_feature_list))

['people', 'tags']

In [6]:
def request_vision(features, image_path, language, smart_crops=''):
    query_params = {
        'api-version': API_VERSION,
        'features': ','.join(features),
        'language': language
    }

    if 'smartCrops' in  features:
        query_params.update({
            'smartcrops-aspect-ratios': smart_crops
        })
    
    endpoint = f'{ENDPOINT}/{MODEL}'
    # method
    headers = {
        'Ocp-Apim-Subscription-Key': API_KEY,
        'Content-Type': 'application/octet-stream'
    }

    # image_path 로 이미지를 바이너리 형태로 읽어서 전송한다.
    with open(image_path, 'rb') as image_file:
        image_data = image_file.read()
    
    response = requests.post(endpoint, params=query_params, headers=headers, data=image_data)

    if response.status_code == 200:
        return response.json()
    else:
        return {}

In [7]:
response1 = request_vision(FEATURES, './data/windows-kitchen.jpg', 'en', '0.75,1.8')
response1

{'modelVersion': '2023-10-01',
 'captionResult': {'text': 'a person using a laptop',
  'confidence': 0.8266631960868835},
 'denseCaptionsResult': {'values': [{'text': 'a person using a laptop',
    'confidence': 0.8266631960868835,
    'boundingBox': {'x': 0, 'y': 0, 'w': 1260, 'h': 473}},
   {'text': 'a person typing on a laptop',
    'confidence': 0.7818875312805176,
    'boundingBox': {'x': 450, 'y': 209, 'w': 360, 'h': 239}},
   {'text': 'a person holding a cup',
    'confidence': 0.7930779457092285,
    'boundingBox': {'x': 813, 'y': 303, 'w': 92, 'h': 79}},
   {'text': 'a woman sitting at a table using a tablet and a cup',
    'confidence': 0.7753849029541016,
    'boundingBox': {'x': 648, 'y': 0, 'w': 583, 'h': 455}},
   {'text': 'a close up of a cell phone',
    'confidence': 0.8052007555961609,
    'boundingBox': {'x': 336, 'y': 383, 'w': 129, 'h': 56}},
   {'text': 'a blurry picture of a radio',
    'confidence': 0.715515673160553,
    'boundingBox': {'x': 301, 'y': 48, 'w': 

In [8]:
def draw_image(image_path, features, response_json):
    image = Image.open(image_path)
    draw = ImageDraw.Draw(image)
    font = get_font()

    # boundingBox 그려주는 코드가 추가되어야 함
    for feature in features:
        name = '{}Result'.format(feature)
        # print(name, response_json[name])
        result_object = response_json[name]
        color = random_color()

        if 'values' in result_object.keys():
            block_lst = result_object['values']

            for block in block_lst:
                bounding_box = block.get('boundingBox', None)
                confidence = block.get('confidence', 1)

                if bounding_box and confidence > 0.8:
                    x, y, w, h = bounding_box['x'], bounding_box['y'], bounding_box['w'], bounding_box['h']
                    draw.rectangle([(x, y), (x + w, y + h)], outline=color, width=5)
                    draw.text((x + 10, y + 10), text=feature, fill=color, font=font)

    return image

In [9]:
with gr.Blocks() as demo:
    # def upload_image(image_path):
    #     print('upload image', image_path)
    
    # def input_image(image_path):
    #     print('image image', image_path)

    def change_image(image_path, features, language, smart_crops):
        # print('change image', image_path)
        if image_path:
            response_json = request_vision(features, image_path, language, smart_crops)
            
            # print(response_json)
            # for feature in features:
            #     print(response_json['{}Result'.format(feature)])
            
            image = draw_image(image_path, features, response_json)
            return image, response_json
        else:
            return None, None
    
    def change_features(features):
        # print(features)
        # features 를 받아서 value로 세팅
        if 'smartCrops' in features:
            return features, gr.update(visible=True)
        else:
            return features, gr.update(visible=False)
    
    def change_language(language, features):
        print(language)
        print(features, FEATURES[:4])
        print(set(features), features)
        # features 를 받아서 value로 세팅
        value = list(set(FEATURES[:4]) & set(features))
        if 'ko' == language:
            return language, gr.update(choices=FEATURES[:4], value=value)
        else:
            return language, gr.update(choices=FEATURES, value=value)

    # 언어선택, features, smartCrops 값
    with gr.Column():
        language_radio = gr.Radio(label='언어 선택', choices=['en', 'ko'], value='en')
        features_checkbox = gr.CheckboxGroup(label='Features', choices=FEATURES)
        smart_crops_textbox = gr.Textbox(label='Smart Crops')

    # 입력 이미지
    with gr.Column():
        input_image = gr.Image(label='입력 이미지', type='filepath', height=400)
        send_button = gr.Button('전송')

    # 출력 이미지, JSON 형태
    with gr.Row():
        output_image = gr.Image(label='출력 이미지', type='pil', interactive=False)
        output_json = gr.JSON(label='결과 JSON')
    
    # 이벤트
    # input_image.input(input_image, inputs=[input_image], outputs=[])
    # input_image.upload(upload_image, inputs=[input_image], outputs=[])
    input_image.change(change_image, inputs=[input_image, features_checkbox, language_radio, smart_crops_textbox], outputs=[output_image, output_json])
    send_button.click(change_image, inputs=[input_image, features_checkbox, language_radio, smart_crops_textbox], outputs=[output_image, output_json])
    language_radio.change(change_language, inputs=[language_radio, features_checkbox], outputs=[language_radio, features_checkbox])
    features_checkbox.change(change_features, inputs=[features_checkbox], outputs=[features_checkbox, smart_crops_textbox])

demo.launch()

* Running on local URL:  http://127.0.0.1:7860

To create a public link, set `share=True` in `launch()`.


In [10]:
demo.close()

Closing server running on port: 7860
